In [1]:
import random 
import paho.mqtt.client as mqtt
from paho.mqtt import client as mqtt_client
import json
import time
import requests

In [2]:
def keygen(noise, modulus):
    a_key = random.getrandbits((noise ** 2))
 
    while ((a_key % 2) != 1) and (a_key < (modulus ** (noise ** 2) - 1)):
        a_key = a_key + 1
 
    return a_key

def keyGenStronger(noise, modulus):
    """Aims to improve correctness of SHE function"""
    key =0 
    while key<=modulus/2:
        key = keygen(noise, modulus)
        print("New key = ", key)
    return key
    
def encrypt(noise, a_key, a_bit, modulus):
    a_mask          = random.getrandbits(noise)
    a_bit_remainder = a_bit % modulus
 
    while ((a_mask % modulus) != a_bit_remainder):
        a_mask = random.getrandbits(noise)
 
    return a_mask + (a_key * random.getrandbits(noise ** 5))

def decrypt(a_key, a_bit, modulus):
    return (a_bit % a_key) % modulus



# Condition Check

In [3]:
def compair(dec_c,origin_m):
    if dec_c == origin_m:
        print("Correct answer")
    else:
        print("Error")

In [4]:
def breakmessage(m):
    """breaking long int message to 2 messages"""
    
    m_temp = m;
    max_value = 99999998 # accept 8 input digits
    m_div = []
    digit = le8
    i=0
    while m_temp > max_value:
        temp = math.floor(m_temp/digit)
        m_temp = float(m_temp)
        m_low = float(m_temp - temp*digit)
        m_div.append(int(m_low))
        if temp < max_value:
            m_div.append(int(temp))
            
        m_temp = float(temp)
        i = i+1
    return m_div

# SHE Execution

In [5]:
def run_encSHE(key, noise, modulus, m1, m2):
    c1 = encrypt(noise, key, m1, modulus)
    c2 = encrypt(noise, key, m2, modulus)
    print("Cibhertext: ")
    print("c1= ", c1)
    print("c2= ",c2)
    return c1,c2

def run_decSHE(key, modulus, ciphertext_result):
    result = decrypt(key, ciphertext_result, modulus)
    print("Result: ", result)
    return result

# NETPIE Pub/Sub

In [6]:
mqtt_client_name = "AVG"  # from NETPIE device
client_id = "bf384adf-275f-44cf-abcb-17c4c555a23f" # client-ID from NETPIE device
username = "HdzFnV8D7HLQ1PVJyd75DDa7McWeQfrS" # Token from NETPIE device
password = "TUIgG-V2SjCVl5IPc!p~JWi6Ulg*$TBp" # secret from NETPIE device
broker = "broker.netpie.io" # MQTT broker
topic = "@shadow/data/update"
topic_sub = "@shadow/data/updated"
protocol = "mqtt"
PUBLISH_PERIOD = 2000  # milliseconds
port = 1883

In [7]:
def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Connected to NETPIE MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", rc)
    # Set Connecting Client ID
    client = mqtt_client.Client(client_id)
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

def publish_shadow(client,m1,m2):
    msg_dict = {"data": {'c1':m1,'c2':m2,'multiplication':0,'additive':0}}
    msg = json.dumps(msg_dict)
    result = client.publish(topic, msg,qos=0, retain=False)
    
def subscribe(client_id,username):
    url = 'https://api.netpie.io/v2/device/shadow/data'
    basicAuthCredentials = (client_id, username)
    response = requests.get(url, auth=basicAuthCredentials)
    print(response.json())
    return response.json()

In [8]:
## RUN SHE

def run_SHE(m1, m2, noise, modulus):
    key = keyGenStronger(noise, modulus)
    print("key: ", key)
    c1 = encrypt(noise, key, m1, modulus)
    c2 = encrypt(noise, key, m2, modulus)
    print(f"ciphertext c1 : {c1} \n ciphertext {c2}")
    return c1, c2, key

In [19]:
# def run_NETPIE(c1, c2):
#     client = connect_mqtt()
#     client.loop_start()  # start loop
#     publish_shadow(client, c1, c2)  # publish data to @shadow
#     time.sleep(4)
#     client.loop_stop()
#     result = subscribe(client_id, username)
    
def run_NETPIE(c1,c2):
    client = connect_mqtt()
    client.loop_start() # Start loop
    publish_shadow(client,c1,c2) # publish data to @shadow/data/update
    time.sleep(4)
    client.loop_stop()
    result = subscribe(client_id,username)
    
    return result

# Results

In [ ]:
noise = 2
modulus = 4

In [35]:
m1 = 1
m2 = 2

c1,c2,key = run_SHE(m1, m2, noise, modulus)

New key =  15
key:  15
ciphertext c1 : 954027931 
 ciphertext 3787147832


In [36]:

result = run_NETPIE(c1,c2)
# print(result)

Connected to NETPIE MQTT Broker!
Connected to NETPIE MQTT Broker!
{'deviceid': 'a5a2ec78-be5e-4f8a-af21-2158855434b8', 'data': {'additive': 4741175763, 'c1': 954027931, 'c2': 3787147832, 'multiplication': 3613044810554095600}, 'rev': 33, 'modified': 1651120921864}


In [37]:
c_netpie_add = result["data"]["additive"]
c_netpie_mul = result["data"]["multiplication"]

print(f"m_netpie_add = {c_netpie_add} \n c_netpie_mul = {c_netpie_mul}")
m_NETPIE_add = run_decSHE(key, modulus, c_netpie_add)
m_NETPIE_mul = run_decSHE(key, modulus, c_netpie_mul)

m_netpie_add = 4741175763 
 c_netpie_mul = 3613044810554095600
Result:  3
Result:  2


In [38]:
def compare(dec_c,origin_m):
    if dec_c == origin_m:
        print("Correct answer")
    else:
        print("Error")

In [39]:
print("Compare the result from NETPIE")
compare(m_NETPIE_add, m1+m2)
compare(m_NETPIE_mul, m1*m2)

Compare the result from NETPIE
Correct answer
Correct answer
